<h2><font size = 5>Peer-graded Assignment:
Segmenting and Clustering Neighborhoods in Toronto</font></h2>

## Part 3

In [1]:
# Import necessary Libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import html5lib
from bs4 import BeautifulSoup
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


#### 2. Scrape the provided wiki page to obtain the list of postal codes
> `https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M`

#### Ensure ability to establish connection to the URL and receive a response

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal_url = requests.get(url)
postal_url

<Response [200]>

#### Get relevant part of datafile and transform it into a *pandas* dataframe

In [3]:
postal_data = pd.read_html(postal_url.text)
len(postal_data), type(postal_data)

(3, list)

From above printout of length and type, there are 3 tables with a type list

In [4]:
# Pull the relevant datatable
postal_data = postal_data[0]
postal_data.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


<h4>Define information of interest and filter dataframe,
removing the Borough's that read <em>'Not assigned'</em></h3>

In [5]:
# keep only columns that include an assigned borough, and ignore if unassigned
filtered_columns = postal_data[~postal_data.Borough.str.contains('Not assigned')]
filtered_columns.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Reading the <em>Shape</em> of the dataframe to print the number of rows

In [6]:
filtered_columns.shape


(103, 3)

#### Create a geocoder function to loop through the postal codes and assign the correct longitude and latitude

In [7]:
import geocoder

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while lat_lng_coords is None :
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    return latitude, longitude

#### Read the cvs file data into a dataframe

In [8]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the latitude and longitude coordinates with the corresponding Postal Code in the initial dataset

In [9]:
merge_geo_data = pd.merge(geo_df, filtered_columns, on='Postal Code')
geo_data = merge_geo_data[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
geo_data.sort_index(kind='quicksort')
geo_data.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Obtain all the borough's in Toronto

In [10]:
toronto_data = geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Define Foursquare Credentials and Version


Check the dataset to find the number of boroughs and neighborhoods

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


#### Importing the User Client Credentials for FourSquare

In [12]:
CLIENT_ID = '23XKBQIHR43GO4QC5TUCSO2QLXJCLTVZDKRH5M4GIQXAS0HU' # your Foursquare ID
CLIENT_SECRET = 'Z2RELZ12VGKJFGAO0E2P1NDLNA2ZZSJIC4JRQNYE0R5PXHAU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: 23XKBQIHR43GO4QC5TUCSO2QLXJCLTVZDKRH5M4GIQXAS0HU
CLIENT_SECRET:Z2RELZ12VGKJFGAO0E2P1NDLNA2ZZSJIC4JRQNYE0R5PXHAU


#### Obtaining the coordinates specific to Toronto, Ontario

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating a map of Toronto with neighborhoods superimposed on top

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to the map
for lat, lon, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(toronto_data, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9933FF',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto


### Explore Neighborhoods in Toronto

#### Creating a function to repeat the same process to all the neighborhoods in Toronto

In [15]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):

    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = \
            'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return nearby_venues

#### Using the above function run code on each neighborhood and create a new dataframe called *toronto_venues*.

In [16]:
toronto_venues = get_nearby_venues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

#### Check the size of the resulting dataframe

In [17]:
print(toronto_venues.shape)
toronto_venues.head()


(872, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


Finding the number venues were returned for each neighborhood along with number of unique Categories

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.groupby('Neighborhood').count()

There are 195 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


### Analyzing the Neighborhoods using one-hot encoding

In [19]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Examining the new dataframe size
toronto_onehot.shape


(872, 195)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0


#### Creating a dataframe for each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

    temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

#### Displaying a dataframe for the top 10 venues for each neighborhood

In [23]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Bistro,Restaurant,Bakery,Basketball Stadium,Jazz Club,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Performing Arts Venue,Bakery,Coffee Shop,Breakfast Spot,Nightclub,Furniture / Home Store,Climbing Gym,Restaurant,Music Venue
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Comic Shop,Recording Studio,Burrito Place,Restaurant,Brewery,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Modern European Restaurant,Sandwich Place,Bubble Tea Shop,Poke Place,Pizza Place,Miscellaneous Shop


### Making Clusters for Neighborhoods, using *k*-means to cluster the neighborhood into 5 clusters

In [24]:
kcluster = 5

toronto_grouped_cluster = toronto_grouped.drop('Neighborhood', 1)

#run the k-means clustering
kmeans = KMeans(n_clusters=kcluster, random_state=0).fit(toronto_grouped_cluster)

# Check the cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([3, 3, 3, 3, 0, 3, 3, 3, 0, 3])

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last col

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Wine Bar,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Bookstore,Juice Bar,Dessert Shop,Spa
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Pizza Place,Steakhouse,Fast Food Restaurant,Brewery,Restaurant,Board Shop,Coffee Shop,Italian Restaurant,Burrito Place
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Wine Bar,Gay Bar,Pet Store,Park,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Wine Bar,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


### Finally, creating a visualization for the resulting clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kcluster)
ys = [i + x + (i*x)**2 for i in range(kcluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters


## Extras...

#### Cluster 1

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels']
                     == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Italian Restaurant,Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Bookstore,Juice Bar,Dessert Shop,Spa
43,East Toronto,0,Café,Coffee Shop,Bakery,Wine Bar,Gay Bar,Pet Store,Park,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant
47,Central Toronto,0,Dessert Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Coffee Shop,Sandwich Place,Gym,Indian Restaurant,Diner
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Light Rail Station,Restaurant,Burger Joint,Liquor Store,Bank,Bagel Shop,Sports Bar
51,Downtown Toronto,0,Restaurant,Coffee Shop,Italian Restaurant,Café,Bakery,Caribbean Restaurant,Jewelry Store,Diner,Japanese Restaurant,Deli / Bodega
53,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Breakfast Spot,Performing Arts Venue,Chocolate Shop,Restaurant
57,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Modern European Restaurant,Sandwich Place,Bubble Tea Shop,Poke Place,Pizza Place,Miscellaneous Shop
58,Downtown Toronto,0,Coffee Shop,Café,Pizza Place,Hotel,Monument / Landmark,Juice Bar,Seafood Restaurant,Smoke Shop,Bakery,Speakeasy
60,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Sandwich Place,Japanese Restaurant,Museum,Deli / Bodega,Beer Bar,Pub
64,Central Toronto,0,Jewelry Store,Trail,Sushi Restaurant,Park,Gas Station,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop


#### Cluster 2

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels']
                     == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,1,Park,Bus Line,Swim School,Wine Bar,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
50,Downtown Toronto,1,Park,Playground,Trail,Wine Bar,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


#### Cluster 3

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels']
                     == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2,Restaurant,Playground,Wine Bar,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


#### Cluster 4

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels']
                     == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,3,Trail,Health Food Store,Pub,Wine Bar,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
42,East Toronto,3,Park,Pizza Place,Steakhouse,Fast Food Restaurant,Brewery,Restaurant,Board Shop,Coffee Shop,Italian Restaurant,Burrito Place
45,Central Toronto,3,Hotel,Park,Food & Drink Shop,Pizza Place,Breakfast Spot,Gym,Department Store,Sandwich Place,Diner,Dessert Shop
46,Central Toronto,3,Clothing Store,Coffee Shop,Yoga Studio,Café,Seafood Restaurant,Diner,Shoe Store,Restaurant,Rental Car Location,Spa
52,Downtown Toronto,3,Gay Bar,Indian Restaurant,Ice Cream Shop,Café,Hobby Shop,Smoke Shop,Juice Bar,Dance Studio,Beer Bar,Burger Joint
54,Downtown Toronto,3,Café,Coffee Shop,Clothing Store,Theater,Japanese Restaurant,Sandwich Place,Diner,Ramen Restaurant,Shopping Mall,Steakhouse
55,Downtown Toronto,3,Gastropub,Restaurant,Japanese Restaurant,Coffee Shop,Café,Gym,Italian Restaurant,Ice Cream Shop,Creperie,Hotel
56,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Bistro,Restaurant,Bakery,Basketball Stadium,Jazz Club,Breakfast Spot
59,Downtown Toronto,3,Park,Plaza,Café,Hotel,Performing Arts Venue,New American Restaurant,Dance Studio,Lake,Deli / Bodega,Bubble Tea Shop
61,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Hotel,Gluten-free Restaurant,Gym / Fitness Center,Deli / Bodega,Pub,Japanese Restaurant,Bookstore


#### Cluster 5

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels']
                     == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,4,Home Service,Pool,Garden,Coworking Space,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
